In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'vegetables/train'
test_dir = 'vegetables/test'
validation_dir='vegetables/validation'

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = "nearest"
)

validation_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size = 32,
    class_mode = "categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150,150),
    batch_size = 32,
    class_mode = "categorical"
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150,150),
    batch_size = 32,
    class_mode = "categorical"
)


Found 15000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.


In [2]:
from tensorflow.keras import datasets, layers, models


model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)), #nurodom filtrus ir ju dydziu dimensija
    layers.MaxPooling2D(2,2), #sumazinam dimensijas, islaikant svarbiausias savybes
    layers.Conv2D(64, (3,3),  activation='relu'), # antras konvoliucinis sluoksnis
    layers.MaxPooling2D(2,2), #sumazinam dimensijas, islaikant svarbiausias savybes
    layers.Conv2D(128, (3,3),  activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(), # plokstinam duomenis, paversdami is 3d i 1d
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(15, activation='softmax') # isejimo sluoksnis su vienu neuronu, grazinis tikimybe nuo 0 iki 1
])

c:\Users\ostap\OneDrive\Рабочий стол\Mokymai\Duomenu_mokslas\mokymai\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
from tensorflow.keras.optimizers import Adam
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [4]:
print(train_generator.samples)
print(train_generator.batch_size)
print(validation_generator.samples)
print(validation_generator.batch_size)

15000
32
3000
32


In [ ]:
history=model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)


In [ ]:
model.save('vegetables.h5')

In [ ]:
import numpy as np
from PIL import Image

def preprocess_image(image_path):
    img = Image.open(image_path)
    if image_path.endswith(".png"):
        im = Image.open(image_path)
        img = im.convert('RGB')
    img = img.resize((150,150))
    img_array = np.array(img)
    img_array = img_array.astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

class_indices = train_generator.class_indices
class_labels = {value: key for key, value in class_indices.items()}


def predict_vegetable(path_to_img):
    img_array = preprocess_image(path_to_img)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    return class_labels[predicted_class[0]]

# print(predict_vegetable('morka.png'))
# print(predict_vegetable('morka2.jpg'))
# # print(predict_vegetable('pomidoras.jpg'))
# print(predict_vegetable('pomidoras2.jpg'))

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(test_loss)
print(test_accuracy)